In [2]:
import pandas as pd

train_data = pd.read_csv("kg_train.csv/kg_train.csv",encoding='latin-1')

In [3]:
import string
print(string.punctuation)
specialchars = list(string.punctuation)
print(specialchars)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [4]:
import re

patterns = [
re.compile(r'\W'),
re.compile(r'\s+[a-zA-Z]\s+'),
re.compile(r'\s*([0-9])+\s*'),
re.compile(r'\^[a-zA-Z]\s+')]

def clean_text(text):
    
    #Remove all the special characters
    processed_feature = re.sub(patterns[0], ' ', str(text))

    # remove all single characters
    processed_feature= re.sub(patterns[1], ' ', processed_feature)
    
    # remove all single numbers, or all numbers attached to a string
    processed_feature= re.sub(patterns[2], ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(patterns[3], ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    return processed_feature

In [8]:
import nltk
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

wordnet_lemma  = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

In [9]:
train_data.loc[:,'preprocessed_text'] = train_data['text'].apply(clean_text)
train_data

,text,label,preprocessed_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...
1,Will do.,0,will do
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam c know that this proposal migh...
4,fyi,0,fyi
...,...,...,...
5959,I talked to CDM about doing some strategy sess...,0,i talked to cdm about doing some strategy sess...
5960,"Dear Sir/CEO,=20I am Mr.D S Ammer, the purchas...",1,dear sir ceo i am mr s ammer the purchasing di...
5961,Faxing to you now.,0,faxing to you now
5962,"Goodday,I am Joseph vaye,the son of late Issac...",1,goodday am joseph vaye the son of late issac n...


<p> Here we create 'processed_text', which is the result of lemmatizing </p>

In [14]:
list_words = [] 
new_list = []
for doc in train_data['preprocessed_text']:
    for item in doc.split():
        list_words.append(wordnet_lemma.lemmatize(item,get_wordnet_pos(item)))
    new_list.append(' '.join(map(str, list_words)))
    list_words.clear()
train_data['processed_text'] = new_list
train_data

,text,label,preprocessed_text,processed_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...,dear sir strictly private business proposal be...
1,Will do.,0,will do,will do
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...,nora cheryl have email dozen of memo about hai...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam c know that this proposal migh...,dear sir fmadam c know that this proposal migh...
4,fyi,0,fyi,fyi
...,...,...,...,...
5959,I talked to CDM about doing some strategy sess...,0,i talked to cdm about doing some strategy sess...,i talk to cdm about do some strategy session t...
5960,"Dear Sir/CEO,=20I am Mr.D S Ammer, the purchas...",1,dear sir ceo i am mr s ammer the purchasing di...,dear sir ceo i be mr s ammer the purchasing di...
5961,Faxing to you now.,0,faxing to you now,fax to you now
5962,"Goodday,I am Joseph vaye,the son of late Issac...",1,goodday am joseph vaye the son of late issac n...,goodday be joseph vaye the son of late issac n...


<p> Finally, we correct the text </p>

In [21]:
#Write code to check grammar and spelling

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

sub_data_train, sub_data_val, sub_label_train, sub_label_val = train_test_split(train_data, train_data["label"], test_size=0.3, random_state=5)

bow_transformer = CountVectorizer().fit(sub_data_train['processed_text'])

X_train = bow_transformer.transform(sub_data_train['processed_text'])
X_val  = bow_transformer.transform(sub_data_val['processed_text'])

print(X_train.shape)
print(X_val.shape)

(4174, 70658)
(1790, 70658)


In [24]:
#Learn Classifier
clf = MultinomialNB().fit(X_train, sub_label_train)
#Predict Val data
pred_val = clf.predict(X_val)

accuracy = accuracy_score(sub_label_val,pred_val)
print(accuracy)
confusion_matrix(sub_label_val, pred_val)

0.9698324022346368


array([[951,  53],
       [  1, 785]])

In [25]:
data_test = pd.read_csv("kg_test.csv/kg_test.csv",encoding='latin-1')
X_test = bow_transformer.transform(data_test['text'].apply(clean_text))
pred_text = clf.predict(X_test)
submission_file = pd.DataFrame({'Id': data_test.index,'Category':pred_text})
submission_file.to_csv('to_submit.csv',index=False)

# To do's

In [ ]:
#FIRST: Lemmatizing (DONE)

In [ ]:
#SECOND: Spelling and grammar correction

In [ ]:
#THIRD: Implement formula to compute efficiency-accuracy ratio

In [ ]:
#FOURTH: Weird a's show up in dictionary